In [7]:
# https://www.kaggle.com/githubsearch/sentiment-classification-of-tweets

In [8]:
import pandas as pd
import numpy as np
import re, nltk
import csv, string
from string import punctuation
from nltk.corpus import stopwords
from nltk import ngrams
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
stop_words = set(stopwords.words('english'))

In [9]:
#Read Data neg and pos with corresponding label

raw_data_neg = pd.read_csv('./twitter-datasets/train_neg.txt', header=None, sep="\n", encoding='latin1', quoting=csv.QUOTE_NONE, names=['text'],
                               error_bad_lines=False, warn_bad_lines=False).drop_duplicates()
raw_data_neg['text'] = raw_data_neg['text']
raw_data_neg['label'] = 0
raw_data_neg = raw_data_neg

raw_data_pos = pd.read_csv('./twitter-datasets/train_pos.txt', header=None, sep="\n", encoding='latin1', quoting=csv.QUOTE_NONE, names=['text'],
                               error_bad_lines=False, warn_bad_lines=False).drop_duplicates()
raw_data_pos['label'] = 1
raw_data_pos['text'] = raw_data_pos['text']
raw_data_pos = raw_data_pos
    
raw_data = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)

raw_data

,text,label
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,0
1,glad i dot have taks tomorrow ! ! #thankful #s...,0
2,1-3 vs celtics in the regular season = were fu...,0
3,<user> i could actually kill that girl i'm so ...,0
4,<user> <user> <user> i find that very hard to ...,0
5,wish i could be out all night tonight ! <user>,0
6,<user> i got kicked out the wgm,0
7,rt <user> <user> <user> yes she is ! u tell it...,0
8,why is she so perfect <url>,0
9,<user> hi harry ! did u havea good time in aus...,0


# Rename

In [10]:
def cmapRefine(cmap):
    cmap = re.sub('teh', 'the', cmap)
    cmap = re.sub('da', 'the', cmap)
    cmap = re.sub('tha', 'the', cmap)
    
    cmap = re.sub('evar', 'ever', cmap)
    
    cmap = re.sub('likes', 'like', cmap)
    cmap = re.sub('liked', 'like', cmap)
    cmap = re.sub('lk', 'like', cmap)
    
    cmap = re.sub('wierd', 'weird', cmap)
    cmap = re.sub('kool', 'cool', cmap)
    cmap = re.sub('yess', 'yes', cmap)
    cmap = re.sub('pleasee', 'please', cmap)
    cmap = re.sub('soo', 'so', cmap)
    cmap = re.sub('noo', 'no', cmap)
    cmap = re.sub('lovee', 'love', cmap)
    cmap = re.sub('loove', 'love', cmap)
    cmap = re.sub('looove', 'love', cmap)
    cmap = re.sub('loooove', 'love', cmap)
    cmap = re.sub('looooove', 'love', cmap)
    cmap = re.sub('loves', 'love', cmap)
    cmap = re.sub('loved', 'love', cmap)
    cmap = re.sub('wuv', 'love', cmap)
    cmap = re.sub('lov', 'love', cmap)
    cmap = re.sub('luvs', 'love', cmap)
    cmap = re.sub('lurve', 'love', cmap)
    
    cmap = re.sub('lovelove', 'love love', cmap)
    cmap = re.sub('lovelovelove', 'love love love', cmap)
    
    cmap = re.sub('ilove', 'i love', cmap)
    
    cmap = re.sub('liek', 'like', cmap)
    cmap = re.sub('lyk', 'like', cmap)
    cmap = re.sub('likee', 'like', cmap)
    
    cmap = re.sub('mee', 'me', cmap)
    
    cmap = re.sub('sooon', 'soon', cmap)

#     cmap['goodd'] = cmap['gud'] = 'good'
#     cmap['bedd'] = 'bed'
#     cmap['badd'] = 'bad'
#     cmap['sadd'] = 'sad'
#     cmap['madd'] = 'mad'
#     cmap['redd'] = 'red'
#     cmap['tiredd'] = 'tired'
#     cmap['boredd'] = 'bored'
#     cmap['godd'] = 'god'
#     cmap['xdd'] = 'xd'
#     cmap['itt'] = 'it'
#     cmap['lul'] = cmap['lool'] = 'lol'
#     cmap['sista'] = 'sister'
#     cmap['w00t'] = 'woot'
#     cmap['srsly'] = 'seriously'
#     cmap['4ever'] = cmap['4eva'] = 'forever'
#     cmap['neva'] = 'never'
#     cmap['2day'] = 'today'
#     cmap['homee'] = 'home'
#     cmap['hatee'] = 'hate'
#     cmap['heree'] = 'here'
#     cmap['cutee'] = 'cute'
#     cmap['lemme'] = 'let me'
#     cmap['mrng'] = 'morning'
#     cmap['gd'] = 'good'
#     cmap['thx'] = cmap['thnx'] = cmap['thanx'] = cmap['thankx'] = cmap['thnk'] = 'thanks'
#     cmap['jaja'] = cmap['jajaja'] = cmap['jajajaja'] = 'haha'
#     cmap['eff'] = cmap['fk'] = cmap['fuk'] = cmap['fuc'] = 'fuck'
#     cmap['2moro'] = cmap['2mrow'] = cmap['2morow'] = cmap['2morrow'] \
#         = cmap['2morro'] = cmap['2mrw'] = cmap['2moz'] = 'tomorrow'
#     cmap['babee'] = 'babe'
#     cmap['theree'] = 'there'
#     cmap['thee'] = 'the'
#     cmap['woho'] = cmap['wohoo'] = 'woo hoo'
#     cmap['2gether'] = 'together'
#     cmap['2nite'] = cmap['2night'] = 'tonight'
#     cmap['nite'] = 'night'
#     cmap['dnt'] = 'dont'
#     cmap['rly'] = 'really'
#     cmap['gt'] = 'get'
#     cmap['lat'] = 'late'
#     cmap['dam'] = 'damn'
#     cmap['4ward'] = 'forward'
#     cmap['4give'] = 'forgive'
#     cmap['b4'] = 'before'
#     cmap['tho'] = 'though'
#     cmap['kno'] = 'know'
#     cmap['grl'] = 'girl'
#     cmap['boi'] = 'boy'
#     cmap['wrk'] = 'work'
#     cmap['jst'] = 'just'
#     cmap['geting'] = 'getting'
#     cmap['4get'] = 'forget'
#     cmap['4got'] = 'forgot'
#     cmap['4real'] = 'for real'
#     cmap['2go'] = 'to go'
#     cmap['2b'] = 'to be'
#     cmap['gr8'] = cmap['gr8t'] = cmap['gr88'] = 'great'
#     cmap['str8'] = 'straight'
#     cmap['twiter'] = 'twitter'
#     cmap['iloveyou'] = 'i love you'
#     cmap['loveyou'] = cmap['loveya'] = cmap['loveu'] = 'love you'
#     cmap['xoxox'] = cmap['xox'] = cmap['xoxoxo'] = cmap['xoxoxox'] \
#         = cmap['xoxoxoxo'] = cmap['xoxoxoxoxo'] = 'xoxo'
#     cmap['cuz'] = cmap['bcuz'] = cmap['becuz'] = 'because'
#     cmap['iz'] = 'is'
#     cmap['aint'] = 'am not'
#     cmap['fav'] = 'favorite'
#     cmap['ppl'] = 'people'
#     cmap['mah'] = 'my'
#     cmap['r8'] = 'rate'
#     cmap['l8'] = 'late'
#     cmap['w8'] = 'wait'
#     cmap['m8'] = 'mate'
#     cmap['h8'] = 'hate'
#     cmap['l8ter'] = cmap['l8tr'] = cmap['l8r'] = 'later'
#     cmap['cnt'] = 'cant'
#     cmap['fone'] = cmap['phonee'] = 'phone'
#     cmap['f1'] = 'fONE'
#     cmap['xboxe3'] = 'eTHREE'
#     cmap['jammin'] = 'jamming'
#     cmap['onee'] = 'one'
#     cmap['1st'] = 'first'
#     cmap['2nd'] = 'second'
#     cmap['3rd'] = 'third'
#     cmap['inet'] = 'internet'
#     cmap['recomend'] = 'recommend'
#     cmap['ah1n1'] = cmap['h1n1'] = 'hONEnONE'
#     cmap['any1'] = 'anyone'
#     cmap['every1'] = cmap['evry1'] = 'everyone'
#     cmap['some1'] = cmap['sum1'] = 'someone'
#     cmap['no1'] = 'no one'
#     cmap['4u'] = 'for you'
#     cmap['4me'] = 'for me'
#     cmap['2u'] = 'to you'
#     cmap['yu'] = 'you'
#     cmap['yr'] = cmap['yrs'] = cmap['years'] = 'year'
#     cmap['hr'] = cmap['hrs'] = cmap['hours'] = 'hour'
#     cmap['min'] = cmap['mins'] = cmap['minutes'] = 'minute'
#     cmap['go2'] = cmap['goto'] = 'go to'
    return cmap

In [11]:
def processTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub('<url>','',tweet)
    #Convert <use> to AT_USER
    tweet = re.sub('<user>','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

data = pd.DataFrame(data={'text': raw_data['text'].apply(processTweet)})
print(data)

data = data['text'].apply(cmapRefine)

                                                     text
0       vinco tresorpack difficulty object disassemble...
1                            glad dot have taks tomorrow 
2       celtics the regular season were fucked play th...
3                    could actually kill that girl sorry 
4                      find that very hard believe afraid
5                       wish could out all night tonight 
6                                  got kicked out the wgm
7                       yes she tell lips are closed okay
8                                        why she perfect 
9       harry did havea good time aus didnt get see ma...
10      introduction programming with 2nd edition this...
11                                                 white 
12                         dan love and miss you don sad 
13            many wonderful building but still miss you 
14                   annoying because secretly find good 
15      the post boom spanish american fiction suny se...
16      layers

In [12]:
# tokenize helper function
def text_process(raw_text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in list(raw_text) if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords.words('english')]


# tokenize message column and create a column for tokens
data = raw_data.copy()
data['tokens'] = data['text'].apply(text_process) # tokenize style 1
data

,text,label,tokens
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,0,"[vinco, tresorpack, 6, difficulty, 10, 10, obj..."
1,glad i dot have taks tomorrow ! ! #thankful #s...,0,"[glad, dot, taks, tomorrow, thankful, startho]"
2,1-3 vs celtics in the regular season = were fu...,0,"[13, vs, celtics, regular, season, fucked, pla..."
3,<user> i could actually kill that girl i'm so ...,0,"[user, could, actually, kill, girl, im, sorry]"
4,<user> <user> <user> i find that very hard to ...,0,"[user, user, user, find, hard, believe, im, af..."
5,wish i could be out all night tonight ! <user>,0,"[wish, could, night, tonight, user]"
6,<user> i got kicked out the wgm,0,"[user, got, kicked, wgm]"
7,rt <user> <user> <user> yes she is ! u tell it...,0,"[rt, user, user, user, yes, u, tell, lips, clo..."
8,why is she so perfect <url>,0,"[perfect, url]"
9,<user> hi harry ! did u havea good time in aus...,0,"[user, hi, harry, u, havea, good, time, aus, d..."


# Bigrams

In [6]:
def ngrams(input_list):
    #onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams+trigrams
data = data.copy()
data['grams'] = data.tokens.apply(ngrams)
data[['grams']].head()

,grams
0,"[vinco tresorpack, tresorpack 6, 6 difficulty,..."
1,"[glad dot, dot taks, taks tomorrow, tomorrow t..."
2,"[13 vs, vs celtics, celtics regular, regular s..."
3,"[user could, could actually, actually kill, ki..."
4,"[user user, user user, user find, find hard, h..."


In [2]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

In [3]:
print(data[(data.label == 0)][['grams']].apply(count_words)['grams'].most_common(20))
print("\n")
print(data[(data.label == 1)][['grams']].apply(count_words)['grams'].most_common(20))

NameError: name 'data' is not defined

In [6]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))
vectorized_data = count_vectorizer.fit_transform(data['text'])
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

indexed_data

NameError: name 'data' is not defined

In [1]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, data['label'], test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

NameError: name 'indexed_data' is not defined

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear')
clf_output = clf.fit(data_train, targets_train)

In [ ]:
clf.score(data_test, targets_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lf = LogisticRegression(random_state=0, solver='sag', max_iter=150)
lf_output = lf.fit(data_train, targets_train)

In [ ]:
lf.score(data_test, targets_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, max_depth=1000, random_state=0)
rf.fit(data_train, targets_train)

In [ ]:
rf.score(data_test, targets_test)